test the `qpub` command line interface.

In [1]:
    import qpub.main, qpub.base, contextlib, io, git
    import pytest

the first hose tests overlay multiple project forms over top each other. there tests are interactive examples of `qpub`

`qpub` considers three major layout forms:

- flat gist style forms without any directories.
- nested directories.
- nested src file directories.

our documentation goes in the docs folder, and the documentation is built there according to the accepted gitignore convention.

in the beginning of project, we include all tests in the project. the `docs` folder is always tested, except for the built artifacts,
so this is another sink for tests, or tested literature.

* check for entry points
* check for test requirements.

In [2]:
    import pytest

in this test, we'll deploy the same project three different ways using the `qpub` api.

configuring a small project.

In [3]:
    @pytest.fixture
    def project(simple_python_script_project):
        import qpub
        project = qpub.base.Project(repo=simple_python_script_project)
        assert project.repo.working_dir == str(simple_python_script_project)
        assert not project.files
        project.add(
            simple_python_script_project/"sample_project.py",
            simple_python_script_project/"test_project.py"
        )
        assert project.files
        yield project

## configure `flit`

In [4]:
    def test_simple_script_project_flit(project):
        import qpub.flit_
        pyproject = (project.repo.working_dir / qpub.files.PYPROJECT_TOML)
        
        assert not pyproject.exists()
        
        flit = qpub.flit_.Flit(project.repo.working_dir)
        flit.dump()
        data = pyproject.load()
        assert pyproject.exists() and data
        assert "tool" in data and "flit" in data["tool"]
        print(data.get("tool", {}).get("flit", {}).get("metadata", {}).get("requires", []))
        assert ["pandas"] == data.get("tool", {}).get("flit", {}).get("metadata", {}).get("requires", [])
        assert "hypothesis" in data.get("tool", {}).get("flit", {}).get("metadata", {}).get("requires-extra", {}).get("test", [])
        
        

## configure `setuptools`

In [5]:
    def test_simple_script_project_setuptools(project):
        import qpub.setuptools_
        
#         
#         assert not (project.repo.working_dir / qpub.files.SETUP_CFG).exists()
        
#         qpub.setuptools_.Setuptools.dump(project)
#         assert (project.repo.working_dir / qpub.files.SETUP_CFG).exists()


## configure `poetry`

In [6]:
    def test_simple_script_project_poetry(project):
        import qpub.poetry_
        pyproject = (project.repo.working_dir / qpub.files.PYPROJECT_TOML)

        
        poetry = qpub.poetry_.Poetry(project.repo.working_dir)
        poetry.dump()
        
        
        tools = pyproject.load().get("tool", {})
        assert "poetry" in tools